In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
def img_pyramid(img_path):
    image = cv2.imread(img_path)
    lower_reso = cv2.pyrDown(image)
    higher_reso = cv2.pyrUp(image)
    img_path = f'.{img_path.split(".")[1]}'.replace('\\','/')
    cv2.imwrite(f'{img_path}_lower_reso_.png', lower_reso)
    cv2.imwrite(f'{img_path}_higher_reso_.png', higher_reso)

def color_space_conversion(img_path):
    image = cv2.imread(img_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    img_path = f'.{img_path.split(".")[1]}'.replace('\\','/')
    cv2.imwrite(f'{img_path}_hsv_image_.png', hsv_image)

def contrast_adjustment(img_path):
    image = cv2.imread(img_path,0)
    equalized_image = cv2.equalizeHist(image)
    img_path = f'.{img_path.split(".")[1]}'.replace('\\','/')
    cv2.imwrite(f'{img_path}_equalized_image.png', equalized_image)

def Canny_Edge(img_path):
    image = cv2.imread(img_path)
    edges = cv2.Canny(image, 100, 200)
    img_path = f'.{img_path.split(".")[1]}'.replace('\\','/')
    cv2.imwrite(f'{img_path}_edges.png', edges)

def Morphological_Operations(img_path):
    image = cv2.imread(img_path)
    kernel = np.ones((5, 5), np.uint8)
    dilation = cv2.dilate(image, kernel, iterations=1)
    erosion = cv2.erode(image, kernel, iterations=1)
    img_path = f'.{img_path.split(".")[1]}'.replace('\\','/')
    cv2.imwrite(f'{img_path}_dilation.png', dilation)
    cv2.imwrite(f'{img_path}_erosion.png', erosion)

def image_binarization(img_path):
    image = cv2.imread(img_path)
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    img_path = f'.{img_path.split(".")[1]}'.replace('\\','/')
    cv2.imwrite(f'{img_path}_binary_image.png', binary_image)

def CLAHE(img_path):
    image = cv2.imread(img_path)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized_image = clahe.apply(image)
    img_path = f'.{img_path.split(".")[1]}'.replace('\\','/')
    cv2.imwrite(f'{img_path}_equalized_image.png', equalized_image)

def homomorphic_filter(img_path, d0=30, c=1.0, gH=2.0, gL=0.5):
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # 그레이스케일로 이미지 읽기
    if image is None:
        raise ValueError("Image not found or the path is incorrect")

    # Convert image to log domain
    image_log = np.log1p(np.array(image, dtype='float32') / 255)
    
    # Perform DFT
    dft = cv2.dft(image_log, flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)
    
    # Create a high pass filter
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros((rows, cols, 2), np.float32)
    for u in range(rows):
        for v in range(cols):
            duv = np.sqrt((u - crow) ** 2 + (v - ccol) ** 2)
            mask[u, v] = (gH - gL) * (1 - np.exp(-c * (duv ** 2 / d0 ** 2))) + gL

    # Apply mask and inverse DFT
    fshift = dft_shift * mask
    f_ishift = np.fft.ifftshift(fshift)
    img_back = cv2.idft(f_ishift)
    img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])
    
    # Convert back to spatial domain
    img_homomorphic = np.expm1(img_back)
    
    # Clip the values to prevent overflow and normalize
    img_homomorphic = np.clip(img_homomorphic, 0, None)
    img_homomorphic = (img_homomorphic - np.amin(img_homomorphic)) / (np.amax(img_homomorphic) - np.amin(img_homomorphic))
    img_homomorphic = (img_homomorphic * 255).astype(np.uint8)

    img_path = f'.{img_path.split(".")[1]}'.replace('\\','/')

    cv2.imwrite(f'{img_path}_homomorphic_image.png', img_homomorphic)

def add_lighting_effect(img_path, center=None, radius=100, strength=1.5):
    image = cv2.imread(img_path)
    if center is None:
        center = (image.shape[1] // 2, image.shape[0] // 2)
    
    mask = np.zeros_like(image, dtype=np.float32)
    cv2.circle(mask, center, radius, (255, 255, 255), -1)
    mask = cv2.GaussianBlur(mask, (21, 21), 0)
    
    highlighted = cv2.addWeighted(image, 1, mask.astype(image.dtype), strength, 0)
    img_path = f'.{img_path.split(".")[1]}'.replace('\\','/')
    cv2.imwrite(f'{img_path}_highlighted_image.png', highlighted)

def Retinex_Algorithm(img_path):
    image = cv2.imread(img_path)
    image = image / 255.0
    sigma = 15
    retinex_image = np.log10(image + 1) - np.log10(cv2.GaussianBlur(image, (0, 0), sigma) + 1)
    retinex_image = (retinex_image - np.min(retinex_image)) / (np.max(retinex_image) - np.min(retinex_image))
    retinex_image = (retinex_image * 255).astype(np.uint8)
    img_path = f'.{img_path.split(".")[1]}'.replace('\\','/')
    cv2.imwrite(f'{img_path}_retinex_image.png', retinex_image)



In [3]:
def image_augmentation(img_path):
    
    img_pyramid(img_path)
    color_space_conversion(img_path)
    contrast_adjustment(img_path)
    Canny_Edge(img_path)
    Morphological_Operations(img_path)
    image_binarization(img_path)
    add_lighting_effect(img_path)
    return Retinex_Algorithm(img_path)
    

In [6]:
def image_augmentation2(img_path):
    return Morphological_Operations(img_path)
    
    

In [7]:
def load_images_and_augmentation(folder):
    images = []
    i = 1
    for filename in os.listdir(folder):
        # 파일의 확장자를 확인하여 이미지 파일만 읽어옵니다.
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(folder, filename)
            print(f'{i}th image augmentation ...ing')
            image_augmentation(img_path)
            i = i + 1

In [8]:
def load_images_and_augmentation2(folder):
    images = []
    i = 1
    for filename in os.listdir(folder):
        # 파일의 확장자를 확인하여 이미지 파일만 읽어옵니다.
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            img_path = os.path.join(folder, filename)
            print(f'{i}th image augmentation ...ing')
            image_augmentation2(img_path)
            i = i + 1

In [10]:

                
    
# 폴더 경로를 지정합니다.
folder_path = './0726_save_img/original'

# 이미지 파일을 읽어옵니다.
res = load_images_and_augmentation2(folder_path)

# 이미지 개수를 출력합니다.
print(f'Done')

1th image augmentation ...ing
2th image augmentation ...ing
3th image augmentation ...ing
4th image augmentation ...ing
5th image augmentation ...ing
6th image augmentation ...ing
7th image augmentation ...ing
8th image augmentation ...ing
9th image augmentation ...ing
10th image augmentation ...ing
11th image augmentation ...ing
12th image augmentation ...ing
13th image augmentation ...ing
Done
